In [ ]:
from langchain.memory import ConversationBufferMemory

# ConversationBufferMemory는 사용하기는 쉽지만 대화가 계속 쌓여서 비용에 좋지 못하다. 
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({'input':'Hi!'},{'output': 'How are you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# memory가 무한정 늘어나는 것을 간단하게 방지하지만, 너무 최근 대화에만 집중한다는 단점이 있음
# 전체 맥락을 보지 못하며, 최근 대화의 개수를 k개로 표현 가능하다.
memory = ConversationBufferWindowMemory(
    return_messages = True,
    k=4
)
def add_message(input, output):
    memory.save_context({"input":input},{"output":output})
    
for n in range(1,7):
    add_message(n,n)
    
memory.load_memory_variables({})

{'history': [HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6')]}

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

## llm을 이용해 memory를 control하는 전략
# 요약이 되기 때문에 훨씬 실용적인 장점
# 단, 비용이 듦
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)
def add_message(input, output):
    memory.save_context({"input":input},{"output":output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("안녕, 나는 김경호야. 나는 한국에 거주하고 있으며 ai 서비스 개발하는 스타트업 회사에서 일하고 있어.","와, 그거 정말 멋지다.")

# add_message("한 가지 고민이 있는데, 우리 회사의 stt 모델을 만들어야 하는데, 성능을 어떻게 하면 개선시킬 수 있을까 싶은 거야","일단 너희 회사가 타겟팅하는 업체의 데이터로 새롭게 훈련을 시켜보는 것은 어떨까?")

get_history()

{'history': [SystemMessage(content='The human introduces themselves as Kim Kyungho, living in Korea and working at a startup developing AI services. The AI responds positively to this information.')]}

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, load_prompt, 

# ChatOpenAI LLM 초기화
# 요약형 메모리를 챗봇에 적용시킨 예시
llm = ChatOpenAI(temperature=0.1)

# Memory 초기화
# 현재는 메모리가 memory.load_memory_variables를 사용했을 때 텍스트를 그대로 내보냅니다.

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=1200,
    memory_key="chat_history"
)

template = """

    당신은 유익한 정보를 제공하는 AI입니다.
    
    {chat_history}
    사람: {question}
    AI:
    
"""

# LLMChain에 메모리와 동적으로 chat_history를 연결
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

# 초기 입력값
question = "나는 AI 서비스를 개발하는 스타트업에서 일하고 있어. STT 성능을 끌어올리는 것을 제1 업무로 받았는데 쉽지가 않아 조언을 해줘"
chain.predict(question=question)




> Entering new LLMChain chain...
Prompt after formatting:


    당신은 유익한 정보를 제공하는 AI입니다.
    
    
    사람: 나는 AI 서비스를 개발하는 스타트업에서 일하고 있어. STT 성능을 끌어올리는 것을 제1 업무로 받았는데 쉽지가 않아 조언을 해줘
    AI:
    


> Finished chain.


'STT(음성 인식) 기술을 개선하는 것은 매우 중요한 일이며, 몇 가지 조언을 드릴게요.\n\n1. 데이터 수집 및 정제: 다양한 음성 데이터를 수집하고 정제하여 모델의 학습에 활용하세요. 또한, 데이터의 다양성과 양을 고려해야 합니다.\n\n2. 모델 개선: 최신의 딥러닝 기술을 활용하여 모델을 개선하고 성능을 향상시키세요. Transfer learning이나 fine-tuning 기술을 사용하여 모델을 최적화할 수 있습니다.\n\n3. 오류 분석: STT 시스템이 어떤 오류를 발생시키는지 분석하고 해당 부분을 개선하는 방향으로 작업을 진행하세요. 오류를 분석하여 모델을 보완하는 것이 중요합니다.\n\n4. 테스트 및 검증: 개선된 모델을 다양한 데이터셋을 활용하여 테스트하고 검증하세요. 이를 통해 모델의 성능을 확인하고 필요한 조치를 취할 수 있습니다.\n\n이러한 단계를 차근차근 진행하면 STT 성능을 향상시킬 수 있을 것입니다. 힘내세요!'

In [13]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

# ChatOpenAI LLM 초기화
# 요약형 메모리를 챗봇에 적용시킨 예시
llm = ChatOpenAI(temperature=0.1)

# Memory 초기화
# 현재는 메모리가 memory.load_memory_variables를 사용했을 때 텍스트를 그대로 내보냅니다.
# 만약 chatbot을 만들거라면, return_messages=True를 설정해야 합니다.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 유익한 정보를 제공하는 ai입니다"),
        MessagesPlaceholder(variable_name='chat_history'),
        ("human","{question}"),
    ]
)


# LLMChain에 메모리와 동적으로 chat_history를 연결
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

# 초기 입력값
question = "나는 AI 서비스를 개발하는 스타트업에서 일하고 있어. STT 성능을 끌어올리는 것을 제1 업무로 받았는데 쉽지가 않아 조언을 해줘"
chain.predict(question=question)




> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 유익한 정보를 제공하는 ai입니다
Human: 나는 AI 서비스를 개발하는 스타트업에서 일하고 있어. STT 성능을 끌어올리는 것을 제1 업무로 받았는데 쉽지가 않아 조언을 해줘

> Finished chain.


'STT(음성 인식) 성능을 향상시키는 것은 중요한 과제입니다. 몇 가지 조언을 드리겠습니다:\n\n1. 데이터 수집 및 전처리: 다양한 환경에서 다양한 사람들의 음성 데이터를 수집하고 전처리하는 것이 중요합니다. 이를 통해 모델의 일반화 성능을 향상시킬 수 있습니다.\n\n2. 모델 최적화: 최신의 딥러닝 모델을 사용하여 성능을 향상시키세요. Transfer learning, data augmentation, regularization 등의 기술을 활용하여 모델을 최적화하세요.\n\n3. 환경 및 잡음 처리: 다양한 환경에서 발화되는 음성을 처리할 수 있는 모델을 개발하세요. 잡음 제거 및 환경 속성을 고려한 모델을 구축하여 성능을 향상시킬 수 있습니다.\n\n4. 사용자 피드백 반영: 사용자들의 피드백을 수집하고 모델을 개선하는 데 활용하세요. 사용자들이 자주 발화하는 어휘나 문장을 분석하여 모델을 업데이트하세요.\n\n5. 지속적인 테스트 및 개선: STT 모델의 성능을 지속적으로 테스트하고 개선하는 프로세스를 구축하세요. 새로운 기술이나 방법론을 적용하여 성능을 끌어올리세요.\n\n이러한 조언을 참고하여 STT 성능을 향상시키는 데 도움이 되기를 바랍니다.'

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

# ChatOpenAI LLM 초기화
# 요약형 메모리를 챗봇에 적용시킨 예시
llm = ChatOpenAI(temperature=0.1)

# Memory 초기화
# 현재는 메모리가 memory.load_memory_variables를 사용했을 때 텍스트를 그대로 내보냅니다.
# 만약 chatbot을 만들거라면, return_messages=True를 설정해야 합니다.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 유익한 정보를 제공하는 ai입니다"),
        MessagesPlaceholder(variable_name='chat_history'),
        ("human","{question}"),
    ]
)
# INPUT이 없으면 인자에 대한 오류가 납니다. 
def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain =  RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
            'question':question
        }
    )
    memory.save_context(
        {"input":question},
        {"output":result.content}
    )
    print(result)


In [4]:
question="주니어 ai 개발자로 스타트업에 입사했습니다. 파이썬과 tensorflow, pytorch을 통한 모델링, mysql을 통한 db 관리 등을 할 수 있습니다. 어떤 것을 더 공부하면 시니어 개발자가 될 수 있을까요?"
invoke_chain(question=question)


content='시니어 개발자가 되기 위해서는 다양한 기술과 경험을 쌓아야 합니다. 여러분이 이미 파이썬, TensorFlow, PyTorch, MySQL 등을 다룰 수 있다는 것은 훌륭한 시작입니다. 시니어 개발자가 되기 위해 고려해볼만한 몇 가지 포인트는 다음과 같습니다:\n\n1. 깊은 이해와 전문성: 이미 다뤄보신 기술들에 대해 깊은 이해를 갖고 전문성을 키우는 것이 중요합니다. 해당 기술들의 내부 동작 원리, 최적화 방법, 고급 기능 활용 등을 공부하고 실무 경험을 통해 실력을 향상시키세요.\n\n2. 소프트웨어 아키텍처 및 디자인: 시니어 개발자는 시스템 전체를 이해하고 설계할 수 있는 능력이 필요합니다. 소프트웨어 아키텍처, 디자인 패턴, 코드 품질 등에 대한 지식을 쌓고 프로젝트를 리드해보세요.\n\n3. 팀 리더십 및 커뮤니케이션: 시니어 개발자는 팀을 이끄는 역할도 수행할 수 있어야 합니다. 팀원들과의 원활한 커뮤니케이션, 리더십 능력, 프로젝트 관리 능력을 키우는 것이 중요합니다.\n\n4. 새로운 기술 습득: 빠르게 변화하는 기술 트렌드에 대한 이해와 새로운 기술을 습득하는 능력이 시니어 개발자에게 요구됩니다. 지속적인 학습과 자기계발을 통해 기술적 역량을 강화하세요.\n\n이러한 요소들을 고려하여 기존 기술을 더 깊이 있게 공부하고, 새로운 기술 및 역량을 습득하며 경험을 쌓아가면 시니어 개발자로 성장할 수 있을 것입니다. 계속해서 열정을 가지고 노력하시면 좋은 결과를 얻을 수 있을 것입니다.'
